Realizando imports e download de stopwords

In [ ]:
import ir_datasets
import nltk

import logging
import sys
import json

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

nltk.download('stopwords')

In [ ]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                    format='[%(asctime)s]%(levelname)s(%(name)s): %(message)s')
LOGGER = logging.getLogger(__name__)

Inicializando todos os dicts

In [ ]:
map_docs = {}
map_queries = {}
map_qrels = {}
map_index = {}

Importando o dataset via ir_datasets

In [ ]:
dataset = ir_datasets.load("cord19/trec-covid")

Processando map de consultas

In [ ]:
stop_words = set(stopwords.words('english'))

for query in dataset.queries_iter(): 
    text = f'{query.title} {query.description}'
    processed_text = word_tokenize(text.lower())
    processed_text = ' '.join([word for word in processed_text if word.isalpha() and word not in stop_words])
    LOGGER.info(f'Query ID: {query.query_id} \nText: {processed_text}\n')
    map_queries[query.query_id] = processed_text


In [ ]:
map_queries

Processando map com os documentos
 * Alterando indices para integer
 * Fazendo um map com os indices para converter lista de relevância depois

In [ ]:
i = 1
for doc in dataset.docs_iter():
    text = f'{doc.title} {doc.abstract}'
    processed_text = word_tokenize(text.lower())
    processed_text = ' '.join([word for word in processed_text if word.isalpha() and word not in stop_words])
    map_docs[i] = processed_text
    map_index[doc.doc_id] = i
    i += 1
    

In [ ]:
map_docs

In [ ]:
map_index

Processando map com os dados de relevância das consultas
* Salvando, para cada consulta, ID da consulta, ID do doc e sua relevância 

In [ ]:
map_qrels = {}

In [ ]:
for qrel in dataset.qrels_iter():
    doc_index = map_index.get(qrel.doc_id, None)
    if doc_index is not None:
        map_qrels[qrel.query_id] = map_qrels.get(qrel.query_id, {})
        doc_index = map_index[qrel.doc_id]
        map_qrels[qrel.query_id][doc_index] = qrel.relevance

In [ ]:
# Salvar, para cada consulta, um array com os documentos cuja relevância é maior que 0
for query_id, qrels in map_qrels.items():
    map_qrels[query_id] = [doc_index for doc_index, relevance in qrels.items() if relevance == 2]

In [ ]:
map_qrels

Salvando os maps em JSON para utilizar nos próximos notebooks

In [ ]:
with open('map_qrels.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_qrels,          # Dados a serem salvos
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhor legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )

In [ ]:
with open('map_queries.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_queries,        # Dados a serem salvos
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhor legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )

In [ ]:
with open('map_docs.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_docs,           # Dados a serem salvos
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhor legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )